In [14]:
import numpy as np
import astropy.units as u

from SSLimPy.LIMsurvey import higherorder as sho
from SSLimPy.utils import utils as utl

In [2]:
k1 = 0.05
k2 = 0.1

In [ ]:
print(sho.vF2(k1, 1, k2, 0.4, np.pi))
print(sho.vF2(k2, 0.4, k1, 1, -np.pi))

In [ ]:
utl._addVectors(k2, 0.4, 0.2,
        k2, -0.4, np.pi + 0.2,
        )

In [ ]:
sho.vF3(k1, -1, np.pi,
        k2, 0.4, 0.2,
        k2, -0.4, np.pi + 0.2,
        )

In [6]:
import SSLimPy.interface.sslimpy as sslimpy
from SSLimPy.LIMsurvey import PowerSpectra

In [ ]:
cosmodict={"h":0.67,"Omegam":0.32,"Omegab":0.04,"As":2.1e-9, "mnu":0.06}
settings = {"code":"class",
            "do_RSD" : True,
            "nonlinearRSD" : True,
            "FoG_damp" : "ISTF_like",
            "halo_model_PS" : True,
            "Smooth_window" : True,
            }
Asslimpy = sslimpy.sslimpy(settings_dict=settings,
                           cosmopars=cosmodict)

pobs = PowerSpectra.PowerSpectra(Asslimpy.fiducialcosmo,Asslimpy.fiducialastro)

In [8]:
k = pobs.k.value
pk = pobs.cosmology.matpow(pobs.k,0).value

In [ ]:
sho.TrispectrumL0(k1, 1, 0,
                  k1, -1, np.pi,
                  k2, 0.4, 0.2,
                  k2, -0.4, np.pi + 0.2,
                  k, pk)

In [20]:
import matplotlib.pyplot as plt


In [ ]:
mu1 = np.linspace(-1,1,200)
mu2 = np.linspace(-1,1,7)
Tmu = np.empty_like(mu1)
for mui2 in mu2:
    for imu, mui in enumerate(mu1):
        Tmu[imu] = sho.TrispectrumL0(k1, mui, 0, k1, -mui, np.pi, k2, mui2, 0.2, k2, -mui2, np.pi + 0.2, k, pk)
    plt.plot(mu1, Tmu)

In [ ]:
phi1 = np.linspace(-np.pi,np.pi,200)
Tmu = np.empty_like(phi1)
Tmuflipped = np.empty_like(Tmu)
for iph, phi in enumerate(phi1):
    Tmu[imu] = sho.TrispectrumL0(k1, 0, phi, k1, 0, np.pi + phi,
                                 k2, 0.2, 0.2, k2, -0.2, np.pi + 0.2,k, pk)
    Tmuflipped[imu] = sho.TrispectrumL0(k1, 0, np.pi + phi, k2, 0.2, 0.2,
                                        k2, -0.2, np.pi + 0.2, k1, 0, phi,
                                        k, pk)
plt.plot(phi1, Tmu)
plt.plot(phi1, Tmuflipped)


In [32]:
import SSLimPy.LIMsurvey.Covariance as sco

In [12]:
cov = sco.nonGuassianCov(pobs)

In [ ]:
cov._integrate_4h(k1 * u.Mpc**-1,k2 * u.Mpc**-1)